In [1]:
import sys 
sys.path.insert(0, '/Users/deliaahmed/Desktop/MLE-9/CAPSTONE/4thBrain_XAI/')
#sys.path.insert(1, '/Users/deliaahmed/Desktop/MLE-9/CAPSTONE4thBrain_XAI/BaseModel')


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
from fastai import *
from fastai.vision import *
from fastai.callback import *
from random import randint
#from data_loader import *
#from Functions.data_loader import *
#from lib.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric
#from lib.object_detection_helper import *
#from BaseModel.RetinaNetFocalLoss import RetinaNetFocalLoss
#from BaseModel.RetinaNet import RetinaNet
#from lib.sampling import get_slides
from Functions.sampling import get_slides


In [12]:

size=512
path = Path('./')

database = Database()
#database.open(str(path/'./Databases/MITOS_WSI_CMC_MEL.sqlite'))
database.open(str('/Users/deliaahmed/Desktop/MLE-9/CAPSTONE/4thBrain_XAI/Database/MITOS_WSI_CMC_MEL.sqlite'))

slidelist_test_1 = ['14','18','3','22','10','15','21']
slidelist_test_2 = ['1','20','17','5','2','11','16']
slidelist_test_3 = ['13','7','19','8','6','9', '12']
slidelist_test = slidelist_test_1

## Split dataset into train/validation and test on slide level

Convert database into in-memory object

In [14]:
lbl_bbox, train_slides,val_slides,files = get_slides(slidelist_test=slidelist_test, negative_class=1,  size=size,database=database,basepath='/Volumes/DA_HDD/CAPSTONE/WSI/')

[autoreload of Functions.sampling failed: Traceback (most recent call last):
  File "/Users/deliaahmed/anaconda3/envs/py39_12/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/deliaahmed/anaconda3/envs/py39_12/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/deliaahmed/anaconda3/envs/py39_12/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/deliaahmed/Desktop/MLE-9/CAPSTONE/4thBrain_XAI/Functions/sampling.py", line 9, in <module>
    class SlideContainer():
  File "/Users/deliaahmed/Desktop/MLE-9/CAPSTONE/4thBrain_XAI/Functions/sampling.py", line 11,

NameError: name 'SlideContainer' is not defined

In [ ]:
img2bbox = dict(zip(files, np.array(lbl_bbox)))
get_y_func = lambda o:img2bbox[o]

In [ ]:
bs = 12
train_images = 5000
val_images = 5000

train_files = list(np.random.choice([files[x] for x in train_slides], train_images))
valid_files = list(np.random.choice([files[x] for x in val_slides], val_images))

#train_files = list(np.random.choice(files, train_images))
#valid_files = list(np.random.choice(files, val_images))


In [ ]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )


In [ ]:
train =  ObjectItemListSlide(train_files, path=path)
valid = ObjectItemListSlide(valid_files, path=path)
valid = ObjectItemListSlide(valid_files, path=path)
item_list = ItemLists(path, train, valid)
lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
lls = lls.transform(tfms, tfm_y=True, size=size)
data = lls.databunch(bs=bs, collate_fn=bb_pad_collate_min).normalize()

In [ ]:
data.show_batch(rows=2, ds_type=DatasetType.Train, figsize=(15,15))

In [ ]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7,0.8])


In [ ]:
not_found = show_anchors_on_images(data, anchors)

In [ ]:
crit = RetinaNetFocalLoss(anchors)

In [ ]:
encoder = create_body(models.resnet18, True, -2)
model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=3, sizes=[32], chs=128, final_bias=-4., n_conv=3)



In [ ]:
voc = PascalVOCMetric(anchors, size, [str(i-1) for i in data.train_ds.y.classes[1:]])
learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, ShowGraph], #BBMetrics, ShowGraph
                metrics=[voc]
               )

In [ ]:
learn.split([model.encoder[6], model.c5top5])
learn.freeze_to(-2)


Run learning rate finder

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-4)

In [ ]:
lr=1e-4
learn.fit_one_cycle(10, lr)#, callbacks=[SaveModelCallback(learn, every='improvement', monitor='AP-0', name='model')])

In [ ]:
learn.unfreeze()
lr=1e-4

learn.fit_one_cycle(10, lr)

In [ ]:
learn.unfreeze()
lr=1e-4

learn.fit(30, lr, callbacks=[SaveModelCallback(learn, every='improvement', name='model')])

In [ ]:
slide_object_result(learn, anchors, detect_thresh=0.3, nms_thresh=0.2, image_count=10)

In [ ]:
learn.save('RetinaNet-CMC-MEL-512sh-batch1', with_opt=True)
learn.export("RetinaNet-CMC-MEL-512sh-b1.pth")

In [ ]:
torch.save(learn.model.state_dict(), "RetinaNet-CMC-MEL-512sh-batch1_statedict.pth")

In [ ]:
learn.data.stats